# VTune (Intel VTune Profiler)

- **VTune** es una herramienta de profiling para analizar la performancia de algoritmos en CPUs, GPUs, FPGAs
- Se puede usar a través de la línea de comando o de GUIs en Linux, Windows, Android, etc
- El proceso es estadístico y no se puede esperar el mismo resultado si el proceso se repite. Sin embargo, los patrones se mantienen

Los cuellos de botella típicos que uno puede detectar con VTune incluyen:

- Threading ineficiente
- Baja concurrencia 
- Problemas de acceso de memoria
- I/O wait time
- Instrucciones innecesarias

## Cómo usar VTune?

![alt text](vtune.png)

VTune puede ejecutarse directamente online o desde la línea de comando de DevCloud. En experiencias pasadas, la conexión online de VTune ha fallado sin causa aparente, por ello sugerimos usar la línea de comando. Desafortunadamente en DevCloud no se pueden visualizar directamente los resultados. La alternativa es ejecutar VTune en DevCloud y visualizar los resultados de manera offline en nuestra computadora local. Para ello, el primer paso consiste en instalar VTune en nuestra computadora local. Ello se logra visitando la siguiente página [Descargar VTune](https://www.intel.com/content/www/us/en/developer/tools/oneapi/vtune-profiler-download.html?operatingsystem=window&distributions=webdownload&options=offline), y elegiendo el sistema operativo y el tipo de instalación deseada. Un ejemplo de la configuración de instalación para Windows se muestran en la figura siguiente. Nota: En algunos casos es necesario contar con la última versión de **VTune** para poder visualizar los resultados, por ello es recomendable actualizar la versión si es que ya tienen una descargada. 

![alt text](vtune_desc.png)

Según la figura, estamos escogiendo el tipo de instalador offline que es usado para un caso en el que la conexión de internet no es estable. Luego seguir los pasos de instalación por defecto, además de la configuración recomendada. Nótese que durante la instalación, nos piden instalar complementariamente Microsoft Visual Studio. Por el momento vamos a proceder la instalación sin Visual Studio, y si es necesario, se puede complementar luego de instalar VTune. Luego de instalar, abrimos la interfaz gráfica de VTune. 

![alt text](vtune_desk.png)

La idea con tener VTune instalado en nuestra computadora no es para correr el profiler de manera local, sino es sólo usado para visualizar los resultados obtenidos al ejecutar VTune en DevCloud.

VTune contiene múltiples opciones de análisis (ver imagen en la parte inferior) tanto para CPUs o GPUs. Algunas opciones son únicas para aceleradores GPUs y eso será explorado en la siguiente sección de la clase.

![alt text](options_vtune.png)


## "Hello World" de SYCL en VTune

**Pasos previos**

Vamos a analizar juntos nuestro código de "Hello World" que se encuentra en el archivo fuente `simple.cpp`. Para ello primero vamos a abrir una terminal nueva: 

![alt text](terminal_button.png)

Como podemos observar, el sistema nos hace ingresar por defecto un nodo simple. En la figura inferior, se puede identificar el usuario: `u183985`, y dicho nodo: `s001-n061`. Luego debemos acceder a _otro nodo_ que tenga acceso a GPUs. Podemos acceder a un nodo con GPUs de la forma siguiente (`-I` indica que deseamos acceder al nodo de forma interactiva): `qsub -I -l nodes=1:gen11:ppn=2 -d .`. En algunos casos los nodos de DevCloud se encuentran saturados y se demora mucho en acceder a un nodo de trabajo. Una alternativa es verificar los nodos de DevCloud que están desocupados y acceder directamente a ellos. Para ello podemos usar el comando `pbsnodes | grep -B4 "gen11"`. Una vez que verificamos qué nodos están libres (ejm. `s001-nxxx`), accedemos a ellos de la siguiente forma `qsub -I -l nodes=s001-nxxx:gpu:ppn=2 -d .`. Si nos hemos conectado satisfactoriamente al nodo de DevCloud, deberíamos visualizar lo siguiente:

![alt text](nodo_gpu.png)

Para verificar si nuestro nodo contiene los elementos que requerimos, es decir, verificar si el nodo puede ejecutar programas escritos en **SYCL**, podemos usar el comando `sycl-ls`:

![alt text](sycl_ls_1.png)

Usando los comandos `ls` y `cd` vamos a la carpeta dónde tenemos nuestro archivo fuente `simple.cpp`. Una vez localizado el archivo, usando el comando `pwd` copiamos la ruta del archivo fuente `/home/u183985/Compu_paralela_III/CPAR_GPUs`. Esta fuente es importante y nos servirá para ejecutar **VTune**. Dado que el VTune reconoce los cuellos de botella o potenciales áreas de mejora cuando el código corre un tiempo prudente, vamos a cambiar la constante `N` de archivo fuente a 50. Podemos hacer este cambio usando el comando nano o simplemente con doble click en el archivo y cambiarlo en forma manual.

**Ejecutando VTune - performance-snapshot**

La sintáxis de ejecución de **VTune** incluye el comando `vtune`, seguido de la opción `-collect`, seguido del tipo de análisis que se desea ejecutar `hotspots`, `memory-consumption`, `threading`, etc., seguido finalmente de la ruta del archivo binario que ya hemos obtenido usando `pwd`. Sin embargo, aquí vamos a compilar nuestro achivo fuente usando opciones adicionales de compilación como sigue:

`dpcpp -g -o simple simple.cpp`

En un siguiente paso, vamos a utilizar el `performance snapshot` que nos ayuda a hacer un primer análisis:

`vtune -collect performance-snapshot -- /home/u183985/Compu_paralela_III/CPAR_GPUs/simple`

Al ejecutar este comando vamos a ver muchas un reporte en la terminal que puede llegar a ser no muy amigable para el usuario. Sin embargo, al terminar la ejecución del comando, observaremos que una carpeta denominada `r000ps`. Esta carpeta contiene los resultados de VTune que queremos visualizar.  

![alt text](ps_results.png)

Para visualizar los resultados, utilizamos **VTune** instalado en nuestra computadora local. Para ello vamos a descargar la carpeta y abrir el archivo `r000ps.vtune`. Para descargar esta carpeta, vamos a abrir **Mobaxterm** o cualquier línea de comando que se desea (Para usuarios de Windows se recomienda usar **Mobaxterm** dada que la sintáxis es la misma de Linux y es fácil de usar). Una vez aquí abrimos una ventana nueva y localizamos la carpeta dónde queremos guardar los resultados de **Vtune**:

![alt text](moba_down.png)

Una vez ubicada la carpeta, usamos el comando `scp -r devcloud:`. Aquí `scp -r` es el comando para descargar archivos/carpetas desde un servidor, y el comando `devcloud:` nos permite conectarnos a DevCloud. Para que nuestro sistema reconozca nuestro usuario y carpetas en DevCloud que queremos descargar, agregamos la ruta de la carpeta `r000ps`, y finalizamos con un punto `.`. Un ejemplo de la ruta de descarga se muestra a continuación:

`scp -r devcloud:/home/u183985/Compu_paralela_III/CPAR_GPUs/r000ps .`

![alt text](descarga_moba.png)

**Visualizando los resultados en VTune offline**

Primero abrimos la aplicación offline the **VTune** y vamos a la opción abrir resultados:

![alt text](ps_results_1.png)

En la ventana **Summary** podemos ver marcado en rojo las sugerencias de los siguientes pasos a realizar en **VTune**. **VTune** nos sugiere seguir con un análisis de *Hotspots*, *Microarchitecture Exploration*, *Threading* y *GPU offload* y de esa forma podemos proceder con los siguientes análisis de manera más informada. 

![alt text](summary.png)

**Ejecutando VTune - para análisis específicos**

VTune ofrece muchas opciones de análisis de algoritmos. Algunos ejemplos pueden ser encontrados en la siguiente página web [vtune_analisis](https://www.intel.com/content/www/us/en/develop/documentation/vtune-help/top/command-line-interface/running-command-line-analysis.html). Siguiendo las recomendaciones del `Performance Snapshot`, vamos a ejecutar dos tipos de análisis: i) `gpu-hotspots` y ii) `gpu-offload`:

1. `gpu-hotspots`

Identifica GPUs con alta utilización y estima la efectividad de uso. 

`vtune -collect gpu-hotspots -- /home/u183985/Compu_paralela_III/CPAR_GPUs/simple`

![alt text](gpu_hots.png)

![alt text](gpu_memory_diagram.png)

2. `gpu-offload`

Correlaciona la actividad del código que corre en CPU y GPU e identifica que parte de del algoritmo debe usarse en GPU y que parte en CPU. 

`vtune -collect gpu-offload -- /home/u183985/Compu_paralela_III/CPAR_GPUs/simple`

![alt text](gpu_off.png)

![alt text](gpu_offload.png)

Cómo podemos observar, el programa es muy simple y no podemos observar muchas recomendaciones por parte de Vtune. Por eso, vamos a probar otro algoritmo un poco más complejo para analizarlo.



## Vtune aplicado al MandelBrot

MandelBrot es un problema matemático relacionado a los fractales o a patrones geométricos que se repiten. El resultado de ejecutar este ejercicio puede ser muy interesante a la vista cómo el ejemplo a continuación.

![alt text](mandelbrot.png)

Primero vamos a clonar el siguiente repositorio que contiene un ejemplo de MandelBrot escrito en SYCL por [Intel_mandelbrot](https://github.com/oneapi-src/oneAPI-samples/tree/master/DirectProgramming/C%2B%2BSYCL/CombinationalLogic/mandelbrot):

`git clone https://github.com/oneapi-src/oneAPI-samples.git`

Luego, localizamos la carpeta `mandelbrot` dentro de la carpeta `oneAPI-samples` creada automáticamente al clonar el repositorio:

`cd ~/oneAPI-samples/DirectProgramming/C++SYCL/CombinationalLogic/mandelbrot`

Seguidamente, vamos a crear un archivo denominado `build.sh`. Podemos crearlo desde la línea de comando usando la función `nano` o simplemente creando un archivo de texto desde el *Launcher* o desde jupyter notebooks como se muestra a continuación. Si se tiene curiosidad de revisar cómo está codificado mandelbrot en SYCL, el archivo fuente se encuentra en la carpeta `src` y se denomina `mandel.hpp`. El archivo `main.cpp` es también importante y contiene las instrucciones para correr mandelbrot de forma serial o paralela y contabilizar los tiempos.


In [5]:
%%writefile /home/u183985/Compu_paralela_III/CPAR_GPUs/oneAPI-samples/DirectProgramming/C++SYCL/CombinationalLogic/mandelbrot/build.sh
source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1
mkdir build
cd build
cmake ..
make

Writing /home/u183985/Compu_paralela_III/CPAR_GPUs/oneAPI-samples/DirectProgramming/C++SYCL/CombinationalLogic/mandelbrot/build.sh


Luego vamos a crear otro archivo denominado `run.sh`

In [6]:
%%writefile /home/u183985/Compu_paralela_III/CPAR_GPUs/oneAPI-samples/DirectProgramming/C++SYCL/CombinationalLogic/mandelbrot/run.sh
source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1
cd build
make run

Writing /home/u183985/Compu_paralela_III/CPAR_GPUs/oneAPI-samples/DirectProgramming/C++SYCL/CombinationalLogic/mandelbrot/run.sh


Finalmente, vamos a compilar estos dos archivos, primero el archivo `build` y luego el archivo `run` desde la línea de comando:

`qsub -l nodes=1:gpu:ppn=2 -d . build.sh`

`qsub -l nodes=1:gpu:ppn=2 -d . run.sh`


El resultado del código se puede visualizar en `run.sh.oXXXX`, dónde podemos observar el tiempo que demora el código mandelbrot en correr en forma serial y en forma paralela

![alt text](mandelbrot_ofile.png)

Sin embargo para poder correr Vtune necesitamos el archivo binario del código `mandelbrot.cpp`. Este se encuentra en la carpeta `src` dentro de la carpeta `build` y se denomina `mandelbrot`. Este archivo binario vamos a usar con Vtune. Para ahorrar un poco de tiempo, sólo vamos a realizar el análisis `gpu-hotspots`, para ello ingresamos a un nodo con GPUs si es que no nos encontramos ya en uno:

`qsub -I -l nodes=1:gen11:ppn=2 -d .`

Y luego ejecutamos el comando `gpu-hotspots`:

`vtune -collect gpu-hotspots -- /home/u183985/Compu_paralela_III/CPAR_GPUs/oneAPI-samples/DirectProgramming/C++SYCL/CombinationalLogic/mandelbrot/build/src/mandelbrot`

Una vez finalizada la ejecución de `gpu-hotspots`, procedemos a descargar la carpeta creada que debe tener un nombre del tipo `r00Xgh`. Para ello usamos scp `-r devcloud:/RUTA/NOMBRE de CARPETA .`

Abrimos la carpeta en Vtune y en la opción gráfico o *Graphics*, podremos visualizar una ventana similar a la siguiente:

![alt text](mandelbrot_vhp.png)

También podemos descargar el `mandel.hpp` y visualizar los potenciales hotspots del código:

![alt text](mandelbrot_fuente.png)

En este caso, vtune señala un hotspot esperado que es usado para paralelizar el código. Dado que es un hotspot esperado, no se debería eliminar.

# Referencias:
- https://community.intel.com/t5/Intel-DevCloud/Trouble-selecting-GPU-node-on-DevCloud/m-p/1177749
- https://www.intel.com/content/www/us/en/develop/documentation/vtune-cookbook/top/configuration-recipes/vtune-profiler-server-in-hpc.html
- https://community.intel.com/t5/Intel-DevCloud/Trouble-selecting-GPU-node-on-DevCloud/m-p/1177749
- https://www.intel.com/content/www/us/en/develop/documentation/oneapi-dpcpp-cpp-compiler-dev-guide-and-reference/top/compiler-reference/compiler-options/alphabetical-option-list.html
- https://hpc-wiki.info/hpc/Intel_VTune_Tutorial
- https://www.intel.com/content/www/us/en/developer/tools/oneapi/vtune-profiler.html#gs.dot3vd
- https://www.intel.com/content/www/us/en/develop/documentation/vtune-help/top/analyze-performance/code-profiling-scenarios/python-code-analysis.html 
- https://www.intel.com/content/www/us/en/develop/documentation/vtune-help/top/command-line-interface/running-command-line-analysis.html